In [ ]:
#Read the data files
import pandas as pd

tensorflow_commits = pd.read_csv('C:/Users/aveli/Downloads/tensorflow.csv')
vscode_commits=pd.read_csv('C:/Users/aveli/Downloads/vscode.csv')
react_commits=pd.read_csv('C:/Users/aveli/Downloads/react-native.csv')

total_commits=tensorflow_commits.append(vscode_commits, ignore_index=True)
total_commits=total_commits.append(react_commits, ignore_index=True)
                             

#total_commits.shape

In [ ]:
# Use h2o4gpu if you have it installed

# Add a flag with Default as False. Don't change this unless your kernel uses h2o4gpu.
h2o4gpu_enabled = False

In [ ]:
#Creating various features for each modification

# Total number of lines changed
total_commits['total_changed'] = total_commits['lines_added'] + total_commits['lines_removed']

# Fraction of lines changed per total numbe of lines in file
# We need to account for the fact that new files added with have existing size as '0' and divide by '0' is indeterminate
total_commits['size'].loc[total_commits['size'] == 0] = total_commits['total_changed']
total_commits['ratio_changed'] = total_commits['total_changed'] / total_commits['size']

# Need to weigh the complexity by quantum of change. 
total_commits['rated_complexity'] = total_commits['ratio_changed'] * total_commits['complexity'] * total_commits['total_changed']

# weighing the dmm params by the total changed lines
total_commits['total_dmm_size'] = total_commits['total_changed'] * total_commits['dmm_unit_size']
total_commits['total_dmm_unit_complexity'] = total_commits['total_changed'] * total_commits['dmm_unit_complexity']
total_commits['total_dmm_unit_interfacing'] = total_commits['total_changed'] * total_commits['dmm_unit_interfacing']

# We picked the sqrt of no_of_mod_files to reduce weightage of this feature
total_commits['scaled_rated_complexity']=total_commits['rated_complexity'] * (total_commits['no._of_mod_files'] ** 0.5)

In [ ]:
# Preprocessing the data. ML requires the data to be converted to numericals
#ml_commits = total_commits[['hash','Author', 'no._of_mod_files', 'dmm_unit_size',
#       'dmm_unit_complexity', 'dmm_unit_interfacing', 'complexity', 'functions', 'lines_added', 'lines_removed', 
#       'tokens', 'type']]

ml_commits = total_commits[['hash','Author','total_changed','rated_complexity',
                            'total_dmm_unit_complexity','total_dmm_size','total_dmm_unit_interfacing', 'scaled_rated_complexity']]

# Resetting the frame's index. It is required to retain the integrity of the frame
ml_commits = ml_commits.reset_index().drop(columns = 'index')

# Temporarily dropping text columns for numeric processing
ml_commits_noText = ml_commits.drop(columns = ['Author','hash'])

# Explicitely converting fields to numeric types and filling the NaNs with zeros
ml_commits_numeric = ml_commits_noText.apply(pd.to_numeric,errors ='coerce').fillna(0)

# Adding the Author column back to create a 'total' data frame
ml_commits_total = ml_commits_numeric.copy()
ml_commits_total['Author'] = ml_commits['Author']
ml_commits_total['hash'] = ml_commits['hash']

In [ ]:
# Remove outliers. 
from scipy import stats
import numpy as np

# Calculate z_scores (and if zscore is greater than '3', then its an outlier) and collect normal subset.
ml_commits_nout = ml_commits_total[(np.abs(stats.zscore(ml_commits_total.select_dtypes(exclude='object'))) < 3).all(axis=1)]
ml_commits_nout.to_csv('C:/Users/aveli/Downloads/totalCommits_nout.csv')

# Collect outliers
ml_commits_out = ml_commits_total[~(np.abs(stats.zscore(ml_commits_total.select_dtypes(exclude='object'))) < 3).all(axis=1)]
ml_commits_out.to_csv('C:/Users/aveli/Downloads/totalCommits_out.csv')

In [ ]:
# Applying scaler to regular data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
if (h2o4gpu_enabled == True):
    from h2o4gpu.preprocessing import MinMaxScaler
else:
    from sklearn.preprocessing import MinMaxScaler
    
# Use minMax scaler since this does not distort
# https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02
scaler = MinMaxScaler()
ml_commits_nout_numeric = ml_commits_nout.drop(columns = ['Author','hash'])
data_scaled = scaler.fit_transform(ml_commits_nout_numeric)

In [ ]:
# Try and figure out how many clusters are optimum
# Plot the inertia curve to find the 'elbow'

''' Realised that h2o4GPU has not implemented "Intertia". Hence we will have to use regular Kmeans library for intertia. 
Please ignore this cell entirely.


import h2o4gpu

SSE = []

for cluster in range(1,20):
    kmeans_ss = h2o4gpu.KMeans(n_gpus=1, n_clusters = cluster, init='k-means++', random_state = 42, backend=h2o4gpu)
    %time kmeans_ss.fit(data_scaled)
    SSE.append(kmeans_ss.inertia_)
    print(kmeans_ss.cluster_centers_)
    print(len(SSE))
    print(str(SSE))
    
    # converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,20), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
'''

In [ ]:
# Try and figure out how many clusters are optimum
# Plot the inertia curve to find the 'elbow'
# You don't need to run this for every iteration. Just uncomment and run whenever you need to

# Realised that h2o4GPU has not implemented "Intertia". Hence we will have to use regular Kmeans library for intertia.
# http://docs.h2o.ai/h2o4gpu/latest-stable/h2o4gpu-py-docs/html/_modules/h2o4gpu/solvers/kmeans.html

# Set this to 'True' if you want to plot graph to find elbow
find_elbow = False

if (find_elbow == True): 
    from sklearn.cluster import KMeans

    SSE = []

    for cluster in range(1,20):
        %time kmeans_ss = KMeans(n_clusters = cluster, init='k-means++', random_state = 42)
        kmeans_ss.fit(data_scaled)
        SSE.append(kmeans_ss.inertia_)

    # converting the results into a dataframe and plotting them
    frame = pd.DataFrame({'Cluster':range(1,20), 'SSE':SSE})
    plt.figure(figsize=(12,6))
    plt.plot(frame['Cluster'], frame['SSE'], marker='o')
    plt.xlabel('Number of clusters')
    plt.ylabel('Inertia')

In [ ]:
# Instantiating the KMeans object based on number discovered above
k = 5

from sklearn.mixture import GaussianMixture

# Initializing the Gaussian mixture model 
mix = GaussianMixture(n_components=k,random_state=42)

# Learning the Gaussian mixture model from data   
mix.fit(data_scaled)

# Saving the parameters of Gaussian mixture model in a file
import pickle
vfilename = 'C:/Users/aveli/Downloads/gmm_cluster_model.sav'
pickle.dump(mix, open(vfilename, 'wb'))

# Predicting the cluster labels of the data for the Gaussian mixture model
cluster_frame = pd.DataFrame(data_scaled)
gmm_hash_clusters = mix.predict(cluster_frame)

# Collecting the mean of the Gaussian mixture model in 'gmmcentroids'
gmm_centroids = mix.means_
gmm_covariances = mix.covariances_
combinedCentroids = gmm_centroids[gmm_hash_clusters].sum(axis=1)

# Converting the input data series into pan
ml_commits_nout['Cluster'] = gmm_hash_clusters

In [ ]:
# Covariance matrix heatmap visualization as a sanity check
import seaborn as sns

plt.figure(figsize=(10,10))
sns.set(font_scale=1.5)

# We need as many as cols as we have features
cols=['1','2','3','4','5','6']

# visualization for cluster number
cluster_number = 2
hm = sns.heatmap(gmm_covariances[cluster_number,:,:],
                 cbar=True,
                 annot=True,
                 square=True,
                 fmt='.5f',
                 annot_kws={'size': 12},
                 yticklabels=cols,
                 xticklabels=cols)

plt.title('Covariance matrix of the cluster showing correlation coefficients')
plt.tight_layout()
plt.show()

In [ ]:
# Look at the values of inverted scaling of centroids for sanity
real_centroids = scaler.inverse_transform(gmm_centroids)

# Write these to dataframe
real_centroids_dataFrame = pd.DataFrame(real_centroids, columns=['total_changed','rated_complexity',
                            'total_dmm_unit_complexity','total_dmm_size','total_dmm_unit_interfacing', 'scaled_rated_complexity'])

# Add a cloumn for summing all coloumns
real_centroids_dataFrame['Sum_centroids'] = real_centroids_dataFrame.sum(axis = 1)

#You can write it out as csv if required.
real_centroids_dataFrame.to_csv('C:/Users/aveli/Downloads/totalCommits_centroids.csv')

In [ ]:
# Train the classifier
from xgboost import XGBClassifier
if (h2o4gpu_enabled == True):
    import h2o4gpu as sklearn
else:
    import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Remove text fields before numeric manipulations
ml_commits_nout_numeric_xg = ml_commits_nout.drop(columns=['Author','hash'])

# Prepare the 'X' and 'Y' for the model
X_ml_commits_nout_numeric_xg = ml_commits_nout_numeric_xg.drop(columns = ['Cluster'])
Y_ml_commits_nout_numeric_xg = ml_commits_nout_numeric_xg['Cluster']

# Split the data for 'Training' and 'Testing' datasets
X_train, X_test, y_train, y_test = train_test_split(X_ml_commits_nout_numeric_xg, Y_ml_commits_nout_numeric_xg, random_state=7)

# Instantiate the xgboost model
xgboost_model = XGBClassifier()

# Training the xgboost classifier
xgboost_model.fit(X_train, y_train)

# Predicting the class labels of test data for xgboost classifier
y_pred = xgboost_model.predict(X_test)
 
# check accuracy
accuracy = accuracy_score(y_test, y_pred)
print('accuracy', accuracy)

In [ ]:
if (h2o4gpu_enabled == True):
    from h2o4gpu import metrics
else:
    from sklearn import metrics

metrics.silhouette_score(data_scaled, gmm_hash_clusters)

In [ ]:
# Pickle the XGBoost model
import pickle

# Saving the trained model in a file
filename = 'C:/Users/aveli/Downloads/finalized_model.sav'
pickle.dump(xgboost_model, open(filename, 'wb'))